# Playground: Invoice Processing with Llama Parse and AutoGen
------------------

AI in Back Office and Invoice Processing
Processing invoices is a time-consuming task for many businesses, and it can be prone to errors such as duplicate payments or incomplete data. Machine learning can automate the processing of invoices by extracting data from invoices, verifying the information, and matching it with purchase orders and contracts. This can significantly reduce the time required for invoice processing and free up staff to focus on more high-value tasks.

https://www.artsyltech.com/blog/Machine-Learning-AI-and-the-Future-of-the-Back-Office

# Scope
This notebook uses LlamaParse and autogen for a POC of autmated invoice processing and PO matching. The reuslt of this agentic workflow is agent generated report stored under `tmp/reports`.  

In [2]:
%load_ext autoreload
%autoreload 2

# Basic Setup

In [3]:
import sys
import time
import random
import json
import os
from pathlib import Path  
from IPython.display import Markdown, display

import pandas as pd

In [ ]:
OPENAI_API_KEY = "[OPENAI_API_KEY]" 
LLAMAPARSE_API_KEY = "[LLAMAPARSE_API_KEY]"


In [4]:
import openai
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key  = os.getenv('OPENAI_API_KEY')


# # use chatCompletion model:
# def get_completion(prompt, model="gpt-4-1106-preview", system_message = "You are an helpful Accountant Analyst."): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class

#     system_msg = [{"role": "system", "content": system_message}]
#     user_assistant_msgs  = [{"role": "assistant", "content": prompt[i]} if i % 2 else {"role": "user", "content": prompt[i]} for i in range(len(prompt))]
#     messages = system_msg + user_assistant_msgs

#     completion = openai.chat.completions.create(
#         model=model,
#         messages = messages,
#         temperature=0.1,
#         max_tokens=800,
#         top_p=0.95,
#         frequency_penalty=0,
#         presence_penalty=0,
#         stop=None
#         )
    
#     return completion.choices[0].message.content

In [5]:
from llama_index.llms.openai import OpenAI

llm_gpt4 = OpenAI(temperature=0.1, model="gpt-4")
llm_gpt4_0613 = OpenAI(model_name="gpt-4-0613")
llm_gpt3_turbo_0125 = OpenAI(model_name="gpt-3.5-turbo-0125")

# Invoice Parser Agent

Transform img to pdfs

In [6]:
# import pandas as pd
# from PIL import Image
# import io

# # Load the Parquet file
# file_path = r'C:\Users\dtriepke\Documents\private\AI_Finance_Office\data\invoices\test-00000-of-00001-56af6bd5ff7eb34d.parquet'
# df = pd.read_parquet(file_path)


# # Assuming the image data is stored in a column named 'image_data'
# # Iterate over each row to process images (assuming here it's just one image for simplicity)
# for index, row in df.iterrows():
#     # Get the image data as bytes
#     image_data = row['image.bytes']

#     # Convert bytes data to an image
#     image = Image.open(io.BytesIO(image_data))

#     # Display the image or process it further
#     # image.show()

#     # # Optionally, save the image to disk
#     # image.save(f"data/invoices/output_{index}.png")

#     # Save the image as a PDF file
#     pdf_path = f"data/invoices/output_{index}.pdf"
#     image.save(pdf_path, "PDF", resolution=100.0)



In [7]:
# Uncomment if you are in a Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()
from llama_parse import LlamaParse  # pip install llama-parse

parser = LlamaParse(
    api_key= LLAMAPARSE_API_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown"  # "markdown" and "text" are available
)

# sync
# ocr_processed_document = parser.load_data(".\data\invoices\output_0.pdf")

# async
# documents = await parser.aload_data("./my_file.pdf")

What elements of the invoice data should be delivered? 

In [8]:
from pydantic import BaseModel, Field
# from llama_index.core.bridge.pydantic import Field
from typing import List
from llama_index.program.openai import OpenAIPydanticProgram

from pydantic import BaseModel, Field
from typing import List, Optional

class LineItem(BaseModel):
    """Data model for each line item on an invoice."""
    description: str
    quantity: int
    unit_price: float
    total_price: float

class Invoice(BaseModel):
    """Data model for Invoice"""
    date: str
    due_date: Optional[str] = None
    invoice_number: str
    net_amount: float
    gross_amount: float
    tax_amounts: float
    discounts: Optional[float] = None
    seller_name: str
    creditor_name: str
    purchase_order_number: Optional[str] = None
    payment_terms: Optional[str] = None
    currency: Optional[str] = None
    description_of_goods_or_service: str
    line_items: List[LineItem] = []

# Pydantic program setup continues as previously defined
 

prompt_template_str = """
You are an advanced AI system trained to assist in accounting tasks, particularly in processing invoices for accounts payable. Below is an OCR-processed invoice from our vendor.

{invoice}

Using the information provided in the invoice document, please extract and organize the following details for accounts payable journal entry preparation:

    Vendor Name: 
    Invoice Date: 
    Due Date: 
    Invoice Number: 
    Net Amount (before tax): 
    Gross Amount (including tax): 
    Tax Amounts: (Tax or VAT)
    Discounts: 
    Payment Terms: 
    Purchase Order Number: 
    Currency: 
    Seller Name: 
    Creditor Name (if different from Seller): 
    Description of Goods or Services: 
    Line Item Details:
        - Description:
        - Quantity:
        - Unit Price:
        - Total Price:

If any attribute is missing or not applicable, leave it blank or state 'not applicable'. Please provide the information in a structured and concise manner, ensuring accuracy and clarity.

"""

# Example use of the model with the prompt
invoice_program  = OpenAIPydanticProgram.from_defaults(
    output_cls=Invoice, 
    prompt_template_str=prompt_template_str, 
    verbose=True
)




In [9]:
path_to_invoice = ".\data\invoices\output_0.pdf"
ocr_processed_document = parser.load_data(path_to_invoice)  

# Extract invoice data 
output = invoice_program(invoice=ocr_processed_document)

for f in output.model_fields.keys():
    print(f"{f}: {getattr(output, f)}") 

Started parsing the file under job_id cb77f6f6-cbf0-4c40-9b27-31bf824383a7
Function call: Invoice with args: {"date":"09/18/2015","invoice_number":"97159829","net_amount":889.20,"gross_amount":978.12,"tax_amounts":88.92,"seller_name":"Bradley-Andrade","creditor_name":"Castro PLC","description_of_goods_or_service":"12\" Marble Lapis Inlay Chess Table Top With 2\" Pieces & 15\" Wooden Stand W537","line_items":[{"description":"12\" Marble Lapis Inlay Chess Table Top With 2\" Pieces & 15\" Wooden Stand W537","quantity":2,"unit_price":444.60,"total_price":889.20}]}
date: 09/18/2015
due_date: None
invoice_number: 97159829
net_amount: 889.2
gross_amount: 978.12
tax_amounts: 88.92
discounts: None
seller_name: Bradley-Andrade
creditor_name: Castro PLC
purchase_order_number: None
payment_terms: None
currency: None
description_of_goods_or_service: 12" Marble Lapis Inlay Chess Table Top With 2" Pieces & 15" Wooden Stand W537
line_items: [LineItem(description='12" Marble Lapis Inlay Chess Table Top

# SQL Data base for Purchase Order

In [10]:
import sqlite3

def create_table(cursor, create_table_sql):
    """ Execute a SQL statement to create a table. """
    cursor.execute(create_table_sql)

def insert_data(cursor, insert_sql, data):
    """ Insert data into a table using the provided SQL statement. """
    cursor.executemany(insert_sql, data)

database = 'purchase_orders.db'


INSERT_TEMP_STR = """(po_number, date, line_item_net_amount, total_net_amount, gross_amount, tax_amounts, discounts, creditor_name, description_of_goods_or_service, line_items, line_item_amount) 
        INSERT INTO PurchaseOrders 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """



# with sqlite3.connect(database) as conn:
#     cursor = conn.cursor()

#     # Create the PurchaseOrders table
#     sql_create_purchase_orders_table = """
#         CREATE TABLE IF NOT EXISTS PurchaseOrders (
#             po_number TEXT PRIMARY KEY,
#             date TEXT NOT NULL,
#             line_item_net_amount REAL,
#             total_net_amount REAL,
#             gross_amount REAL,
#             tax_amounts REAL,
#             discounts REAL,
#             creditor_name TEXT,
#             description_of_goods_or_service TEXT,
#             line_items TEXT,
#             line_item_amount REAL
#         ); """
#     create_table(cursor, sql_create_purchase_orders_table)

#     # Insert sample data into PurchaseOrders table
#     insert_purchase_orders_sql = INSERT_TEMP_STR
#     purchase_orders_data = [
#         ('PO1234', '2021-07-15', 200 ,1000.00, 1075.00, 75.00, 0.00, 'ABC Supplies', 'Office Chairs', '5 office chairs', 5),
#         ('PO1235', '2021-07-16', 200, 2000.00, 2150.00, 150.00, 0.00, 'XYZ Corp', 'Desks', '10 office desks', 10)
#     ]
#     insert_data(cursor, insert_purchase_orders_sql, purchase_orders_data)

#     # Commit the changes
#     conn.commit()
#     cursor.close()

# # Output to indicate completion
# print("Purchase orders database has been set up successfully.")



In [11]:
# def insert_purchase_order(data):
#     database = 'purchase_orders.db'
#     with sqlite3.connect(database) as conn:
#         cursor = conn.cursor()
#         sql_insert_query = INSERT_TEMP_STR
#         cursor.execute(sql_insert_query, data)
#         conn.commit()
#         cursor.close()

# purchase_order_data = (
#     333333,  # po_number is None as specified
#     '09/18/2015',  # date
#     444.6, # line item amount
#     889.2,  # net_amount
#     978.12,  # gross_amount
#     '10%',  # tax_amounts
#     None,  # discounts
#     'Bradley-Andrade',  # creditor_name
#     '12" Marble Lapis Inlay Chess Table Top With 2" Pieces & 15" Wooden Stand W537',  # description_of_goods_or_service
#     '2 unite 12" Marble Lapis Inlay Chess Table Top With 2" Pieces & 15" Wooden Stand W537 @ 444.6 each',  # Simplified line_items description
#     2
# )

# # Insert the purchase order into the database
# insert_purchase_order(purchase_order_data)
# print("Purchase order added successfully.")


In [12]:

import sqlite3
from typing import Annotated, Dict, Union

def execute_sql(reflection: Annotated[str, "Think about what to do"],
                sql: Annotated[str, "SQL query"]
                ) -> Annotated[Dict[str, Union[str, list]], "Dictionary with keys 'result' and 'error'"]:
    """
    Execute a SQL query on a SQLite database and return the results or an error message.

    Args:
    reflection (str): Description of what the SQL query is intended to do.
    sql (str): The SQL query to be executed.

    Returns:
    Dict[str, Union[str, list]]: A dictionary containing either the query results under 'result'
                                  or an error message under 'error'.
    """
    database = 'purchase_orders.db'  # Path to the SQLite database file
    conn = sqlite3.connect(database)
    cursor = conn.cursor()

    try:
        # cursor.execute(sql)
        # # For queries that return data
        # if cursor.description:
        #     result = cursor.fetchall()
        # else:
        #     # For queries that modify data/tables
        #     conn.commit()
        #     result = f"Query executed successfully, affected {cursor.rowcount} rows."
        
        # return {"result": result}

        df = pd.read_sql_query(query, conn)
        return df

    except Exception as e:
        return {"error": str(e)}

    finally:
        cursor.close()
        conn.close()

# Example usage of the function
# This should be tailored to actual queries you intend to run.
query = "SELECT * FROM PurchaseOrders LIMIT 5;"  # Example query to fetch data
reflection_text = "Fetch first five purchase orders from the database."

result = execute_sql(reflection_text, query)

result

,po_number,date,line_item_net_amount,total_net_amount,gross_amount,tax_amounts,discounts,creditor_name,description_of_goods_or_service,line_items,line_item_amount
0,PO1234,2021-07-15,200.0,1000.0,1075.00,75.0,0.0,ABC Supplies,Office Chairs,5 office chairs,5.0
1,PO1235,2021-07-16,200.0,2000.0,2150.00,150.0,0.0,XYZ Corp,Desks,10 office desks,10.0
2,333333,09/18/2015,444.6,889.2,978.12,10%,NaN,Bradley-Andrade,"12"" Marble Lapis Inlay Chess Table Top With 2""...","2 unite 12"" Marble Lapis Inlay Chess Table Top...",2.0


In [13]:
# # Table Scheme for SQL querie

# def print_table_schema(cursor, table_name):
#     """
#     Prints the schema of the specified table.
    
#     Args:
#     cursor: SQLite database cursor.
#     table_name (str): The name of the table whose schema is to be printed.
#     """
#     cursor.execute(f"PRAGMA table_info({table_name})")
#     columns = cursor.fetchall()
    
#     # Print column details
#     print(f"Schema of the table '{table_name}':")
#     for col in columns:
#         print(f"Column ID: {col[0]}, Name: {col[1]}, Type: {col[2]}, Not Null: {col[3]}, Default Value: {col[4]}, Primary Key: {col[5]}")

# # Example usage: print the schema of the 'PurchaseOrders' table
# import sqlite3

# # Connect to the SQLite database
# database = 'purchase_orders.db'
# conn = sqlite3.connect(database)
# cursor = conn.cursor()

# print_table_schema(cursor, 'PurchaseOrders')

# cursor.close()
# conn.close()

# RESULT from query above
schema_txt = """
Schema of the table 'PurchaseOrders':
Column ID: 0, Name: po_number, Type: TEXT, Not Null: 0, Default Value: None, Primary Key: 1
Column ID: 1, Name: date, Type: TEXT, Not Null: 1, Default Value: None, Primary Key: 0
Column ID: 2, Name: line_item_net_amount, Type: REAL, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 3, Name: total_net_amount, Type: REAL, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 4, Name: gross_amount, Type: REAL, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 5, Name: tax_amounts, Type: REAL, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 6, Name: discounts, Type: REAL, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 7, Name: creditor_name, Type: TEXT, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 8, Name: description_of_goods_or_service, Type: TEXT, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 9, Name: line_items, Type: TEXT, Not Null: 0, Default Value: None, Primary Key: 0
Column ID: 10, Name: line_item_amount, Type: REAL, Not Null: 0, Default Value: None, Primary Key: 0
"""

In [14]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select)

from llama_index.core import SQLDatabase, VectorStoreIndex
from llama_index.core.objects import SQLTableNodeMapping, ObjectIndex, SQLTableSchema
from llama_index.llms.openai import OpenAI

engine = create_engine(f"sqlite:///{database}")
metadata_obj = MetaData()
# metadata_obj.create_all(engine)
metadata_obj.reflect(engine)

# SQL Wrapper
sql_database = SQLDatabase(engine, include_tables=["PurchaseOrders"])
table_node_mapping = SQLTableNodeMapping(sql_database)

# Create an obejct index for the purchase order db
po_text = f"This table gives the purchase order infomration. Here is the table scheme {schema_txt}" 
table_schema_objs = []
for table_name in metadata_obj.tables.keys():
    print("Add Table Scheme:", table_name)
    table_schema_objs.append(SQLTableSchema(table_name=table_name, context_str=po_text))


# We dump the table schema information into a vector index. The vector index is stored within the context builder for future use.
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

Add Table Scheme: PurchaseOrders


In [15]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)

# Simple version of the SQL engine
# query_engine = NLSQLTableQueryEngine(
#     sql_database=sql_database, 
#     tables=["PurchaseOrders"], 
#     llm=llm_gpt4
# )

# We construct a SQLTableRetrieverQueryEngine. 
# Note that we pass in the ObjectRetriever so that we can dynamically retrieve the table during query-time.
# ObjectRetriever: A retriever that retrieves a set of query engine tools.
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=5),
    llm=llm_gpt4
)


In [16]:
# query_str = """Fetch all purchase orders from the vendor Bradley-Andrade from the purchase_order table.
# Always resturn all fields as a table: (po_number, date, line_item_net_amount, total_net_amount, gross_amount, tax_amounts, discounts, creditor_name, description_of_goods_or_service, line_items, line_item_amount) 
# If a field is empty reply with NA.
# """


# response = query_engine.query(query_str)
# display(Markdown(f"<b>{response}</b>"))

Next I will wrap an pydantic programm over the sql agent.


In [17]:
from pydantic import BaseModel, Field
# from llama_index.core.bridge.pydantic import Field
from typing import List
from llama_index.program.openai import OpenAIPydanticProgram

from pydantic import BaseModel, Field
from typing import List, Optional

class LineItem(BaseModel):
    """Data model for each line item on an invoice."""
    description: str
    quantity: int
    unit_price: float
    total_price: float

class PurchaseOrder(BaseModel):
    """Data model for Invoice"""
    date: str
    net_amount: float
    gross_amount: float
    tax_amounts: Optional[float] = None
    discounts: Optional[float] = None
    seller_name: str
    creditor_name: str
    purchase_order_number: Optional[str] = None
    payment_terms: Optional[str] = None
    currency: Optional[str] = None
    description_of_goods_or_service: str
    line_items: List[LineItem] = []


prompt_template_str = """
You are an advanced AI system trained to assist in accounting tasks, particularly in processing purchase oder for accounts payable. 
Below is a  purchase order from our dabase.

{purchase_order}

Using the information provided in the purcase order documents, please extract and organize the following details for accounts payable journal entry preparation:

    date: str
    net_amount: float
    gross_amount: float
    tax_amounts: Optional[float] = None
    discounts: Optional[float] = None
    seller_name: str
    creditor_name: str
    purchase_order_number: Optional[str] = None
    payment_terms: Optional[str] = None
    currency: Optional[str] = None
    description_of_goods_or_service: str
    line_items: List[LineItem] = []

    Line Item Details:
        - Description:
        - Quantity:
        - Unit Price:
        - Total Price:

If any attribute is missing or not applicable, leave it blank or state 'not applicable'. Please provide the information in a structured and concise manner, ensuring accuracy and clarity.

"""

# Pydantic program setup continues as previously defined 
# Example use of the model with the prompt
purchase_order_program  = OpenAIPydanticProgram.from_defaults(
    output_cls=PurchaseOrder, 
    prompt_template_str=prompt_template_str, 
    verbose=True,
    allow_multiple=True
)


In [18]:
# query_str = """Fetch all purchase orders from the vendor Bradley-Andrade from the purchase_order table.
# Always resturn all fields as a table: (po_number, date, line_item_net_amount, total_net_amount, gross_amount, tax_amounts, discounts, creditor_name, description_of_goods_or_service, line_items, line_item_amount) 
# If a field is empty reply with NA.
# """


# po_request_tmp = lambda query_str: f"""
# Below in tripple quotes is a request from a user to recieve purchase order data:

# '''{query_str}'''

# # Rules
# - Use the purchase_oder.db especually the table 'PurchaseOrders' to answer the request
# - Always resturn all fields as a table: (po_number, date, line_item_net_amount, total_net_amount, gross_amount, tax_amounts, discounts, creditor_name, description_of_goods_or_service, line_items, line_item_amount) 
# - If a field is empty reply with NA.
# """



# po_sql_response = query_engine.query(po_request_tmp(po_request_tmp))
# po_pydantic = purchase_order_program(purchase_order=po_sql_response)

What elements of the sql PO data should be delivered? 

# Bank Account

In [19]:
import datetime

str(datetime.datetime.now())

'2024-06-29 14:39:15.041866'

In [20]:
from pydantic import BaseModel, Field
# from llama_index.core.bridge.pydantic import Field
from typing import List
from llama_index.program.openai import OpenAIPydanticProgram

from pydantic import BaseModel, Field
from typing import List, Optional


# Pydantic object
class BankAccount(BaseModel):
    """Data model for Bank Account"""
    date: str
    net_amount: float




# AutoGen Crew  



## Init config files

In [21]:
import autogen
import os
import openai

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key  = os.getenv('OPENAI_API_KEY')

config_list = [
    {
        "model": "gpt-4o",
        "api_key": os.getenv('OPENAI_API_KEY'),
    }
]

llm_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0.1,
    "config_list": config_list,
    "timeout": 600
}



In [22]:
from llama_index.core.tools import BaseTool 
from pydantic import BaseModel, Field
from typing import Optional, Type, Annotated

def generate_llm_config(tool):
    # Define the function schema based on the tool's args_schema
    function_schema = {
        "name": tool.name.lower().replace(" ", "_"),
        "description": tool.description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    if tool.args is not None:
        function_schema["parameters"]["properties"] = tool.args
    
    if tool.required is not None:
        function_schema["parameters"]["required"].append(tool.required)

    return function_schema


## Build the Invoice processing crew


### Invoice

In [23]:

class InvoiceReader():
    name = "invoice_reader"
    description = "Use this tool extract all data from the invoice" 
    args = {
            "path_to_invoice": {
                "type": "string",
                "description": "The path to the invoice.",
            }
        }
    required = "path_to_invoice"

    def _run(self, path_to_invoice: str):
        """Run invoice reader with given path_to_invoice"""

        # Load parser and process invoice
        ocr_processed_document = parser.load_data(path_to_invoice)  

        # Extract invoice data 
        output = invoice_program(invoice=ocr_processed_document)

        return output

# Instantiate the Invoice
invoice_reader_tool = InvoiceReader()
llm_config_assistant_invoice = {
    # "Seed" : 42,
    "temperature": 0,
        "functions":[
            generate_llm_config(invoice_reader_tool)
        ],
        
    "config_list": config_list,
    "timeout": 120,
}

generate_llm_config(invoice_reader_tool)
# INVOICE
invoice_user = autogen.UserProxyAgent(
    name="invoice_user",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

# Register the tool and start the conversation
invoice_user.register_function(
    function_map={
        invoice_reader_tool.name: invoice_reader_tool._run,
    }
)

# Invoice Assistant 
system_message_invoice_reader = """
You are an expert in scannig and parse invoices.

Responsibilities: Scans and parses invoices, extracting relevant data using the function: `invoice_reader`. 
Don't ask anyone for permission, just proceed with your actions. 
Once you decide that the retrieved invoice data are good enough, you'll forward the final invoice data to the chat manager.
"""
description = "Extracts key data from invoices given a system path."
invoice_extraction_agent = autogen.AssistantAgent(
            name="invoice_extraction_agent",
            llm_config=llm_config_assistant_invoice,
            description= description,
            system_message = system_message_invoice_reader
            
        )


In [24]:
# path = ".\data\invoices\output_0.pdf" 
# invoice_user.initiate_chat( invoice_extraction_agent, 
#                          message = f"Extract the data from invoices '{path}' ", 
#                          clear_history=True)

### PO

In [25]:

class PurchaseOrderReader():
    name = "purchase_order_reader"
    description = "Use this tool extract all data for a purchase order. Use the tool invoice_reader if beforehand to extract vendor information."
    args = {
            "po_request": {
                "type": "string",
                "description": "Purchase order request in text format.",
            }
        }
    required = "po_request"


    def _run(self, po_request: str):
        """Run purchase order reader"""

        po_request_tmp = lambda query_str: f"""
            Below in tripple quotes is a request from a user to recieve purchase order data:

            '''{query_str}'''

            # Rules
            - Use the purchase_oder.db especually the table 'PurchaseOrders' to answer the request
            - Always resturn all fields as a table: (po_number, date, line_item_net_amount, total_net_amount, gross_amount, tax_amounts, discounts, creditor_name, description_of_goods_or_service, line_items, line_item_amount) 
            - If a field is empty reply with NA.
            """


        po_sql_response = query_engine.query(po_request_tmp(po_request))
        po_pydantic = purchase_order_program(purchase_order = po_sql_response)

        return po_pydantic

# Instantiate the Purchase Order
purchase_order_reader_tool = PurchaseOrderReader()
llm_config_assistant_po = {
    # "Seed" : 42,
    "temperature": 0,
        "functions":[
            generate_llm_config(purchase_order_reader_tool)
        ],
        
    "config_list": config_list,
    "timeout": 120,
}

generate_llm_config(purchase_order_reader_tool)

po_user = autogen.UserProxyAgent(
    name="po_user",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

# Register the tool and start the conversation
po_user.register_function(
    function_map={
        purchase_order_reader_tool.name: purchase_order_reader_tool._run
    }
)

# PO Assistant 
system_message= """
You are an expert in managing purchase order.

Responsibilities: Receive Invoice Data from the chat manager and get a matching purchase order (PO) from a database using the tool `purchase_order_reader`. 
The tools request a plane text as parameter.Therefore, use the tool `purchase_order_reader` with a request for feching POs that includes the vendor name.

Don't ask anyone for permission, just proceed with your actions. 
Once you decide that the retrieved PO data are good enough, you'll forward the final PO data to the chat manager.
"""
description = "Extracts a purchase oder given a vendor name."
po_extraction_agent = autogen.ConversableAgent(
    name="po_extraction_agent",
    system_message= system_message,
    description=description,
    llm_config=llm_config_assistant_po,
)



### Bank Account Management


In [26]:

class BankAccountReader():
    name = "bank_account_reader"
    description = "Use this tool extract bank details."
    args = {
            "bank_account_request": {
                "type": "string",
                "description": "Query request for bank account data.",
            }
        }
    required = "bank_account_request"


    def _run(self, bank_account_request: str):
        """Run purchase order reader"""

        ba_request_tmp = lambda bank_account_request: f"""
            Below in tripple quotes is a request from a user to recieve bank account data:

            '''{bank_account_request}'''

            # Rules
            - Always reply with an bank account number and value.
            """

        # TODO: Dummy response
        ba_pydantic = BankAccount(date = str(datetime.datetime.now()), net_amount = 200)

        return ba_pydantic

# Instantiate the Purchase Order
bank_account_reader = BankAccountReader()
llm_config_assistant_ba = {
    # "Seed" : 42,
    "temperature": 0,
        "functions":[
            generate_llm_config(bank_account_reader)
        ],
        
    "config_list": config_list,
    "timeout": 120,
}

generate_llm_config(bank_account_reader)

ba_user = autogen.UserProxyAgent(
    name="ba_user",
    # is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

# Register the tool and start the conversation
ba_user.register_function(
    function_map={
        bank_account_reader.name: bank_account_reader._run
    }
)

# BA Assistant 
system_message= """
You are an expert in managing bank details.

Responsibilities: Receive bank account detais using the tool `bank_account_reader`. 
The tools request a plane text as parameter.Therefore, use the tool `bank_account_reader` with a request for feching bank account details.

Don't ask anyone for permission, just proceed with your actions. 
Once you decide that the retrieved details are good enough, you'll forward the final banking account data to the chat manager.
"""
description = "Extracts a bank account details."
ba_extraction_agent = autogen.ConversableAgent(
    name="ba_extraction_agent",
    system_message= system_message,
    description=description,
    llm_config=llm_config_assistant_ba,
)



### Crew

**Assistants**

In [27]:
po_matching_agent  = autogen.AssistantAgent(
    name="po_matching_agent",
    llm_config=llm_config,
    system_message="""Purchase order matching agent. 
    
    You receive invoice data from 'invoice_extraction_agent' and you receive purchase order data from 'po_extraction_agent'.
    You receive bank balance data from 'ba_extraction_agent'

    Vefify that the invoices details below matches the Purchase Order (PO): 
        line_item_net_amount, 
        total_net_amount, 
        gross_amount, 
        tax_amounts, 
        discounts, 
        creditor_name, 
        description_of_goods_or_service, 
        line_items, 
        line_item_amount
    
    Creates a verification report for the PO-invoice match for a CEO:   
    - Overall Status: green, yellow or red
    - Likelohhod of fraud: number between 1 and 100 that indicates the probability of of fraud as well as the reasonong
    - *Summary*: Summarise the key points.
    - *Red Flags*: Summarise the red flags of missing invoice details in the purchase order  
    - *Check Calculations:* Ensure that all calculations on the invoice, including unit prices, quantities, discounts, and totals, are accurat 
    - Bank account balance coverage: answer can the invoice net amount total be paid with the current bank account balance? 

    The user cannot provide any other feedback or perform any other action. The user can't modify your report. So do not suggest incomplete reports which requires users to modify!

    Here is an example: 
    
        Revised Invoice Verification Report:

        Summary of Findings:
        - Invoice Number: 97159829
        - Creditor Name: Bradley-Andrade
        - Description of Goods or Service: 12" Marble Lapis Inlay Chess Table Top With 2" Pieces & 15" Wooden Stand W537
        - Invoice Date: 09/18/2015
        - Purchase Order Date: 09/18/2015
        - Overall Status: Yellow
        - Likelihood of Fraud: 30% risk (based on discrepancies in tax amounts and missing purchase order number)
        - Bank Account Balance Coverage: No (current balance is $200, invoice net amount is $889.2)
        - Action Items: Contact creditor to resolve tax discrepancy and missing purchase order number; review internal controls.

        Detailed Report:
        - The invoice and purchase order dates align, both being on 09/18/2015.
        - Payment terms are not specified in the provided data for both the invoice and the purchase order.
        - The currency is not mentioned in the provided data; it is assumed to be the local currency of the company.
        - The bank account balance date is 06/23/2024, indicating the financial status at the time of the report.
        - The likelihood of fraud is assessed at 30% risk due to the tax amount discrepancy and the absence of a purchase order number on the invoice. The methodology for this assessment should be reviewed and clarified.
        - The tax amounts are inconsistent, with the invoice stating $88.92 and the purchase order stating $10.00.
        - The purchase order number is missing from the invoice; the corresponding number is 333333.
        - The line item net amount, total net amount, and gross amount match between the invoice and the purchase order.
        - The invoice line item description and amount match the purchase order, with a correction from "2 unite" to "2 units."

        Formatting and Consistency:
        - The report has been formatted for consistency and clarity.

        Next Steps:
        - Contact Bradley-Andrade to address the tax amount discrepancy and to add the missing purchase order number to the invoice.
        - Review internal controls to ensure purchase order numbers are included on all invoices and that tax amounts are correctly calculated and reported.
        - Assess the company's current financial position and take necessary actions to ensure sufficient funds are available to cover upcoming invoices.
            

""",
)

reviewer  = autogen.AssistantAgent(
    name="reviewer",
    llm_config=llm_config,
    system_message="""Reviewer

After the po_matching_agent writes the a report, it's passed to you. Your primary role is to ensure the report's quality and efficiency. 
You are responsible for review the report form not the report itself. Provide feedback to the po_matching_agent.
Iterate until po_matching_agent writes report that is perfect. You decide that the report is successful or not.

""",
description = "You review invoice verification report from the po_macthing_agent and execute code to store the report. When you forward the python code say 'TERMINATE'"
)

logger  = autogen.AssistantAgent(
    name="logger",
    llm_config=llm_config,
    system_message="""Logger and Python Code Developer

Recieve the Report from Reviewer and write a python code to store the report as *.txt file with the invoive number as file name with report_directory = "./reports"  

In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute. 
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself. 
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly. 
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill. 
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user. Include no more than one code block in a response. 
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user. 
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try. 
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible. 
Please conclude with "TERMINATE" once you have successfully answered the user's instruction or question.

""",
description = "Write python code to store report as txt files."
)

Initializer = autogen.UserProxyAgent(
    name="init",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)



**Group Chat**

In [28]:
groupchat = autogen.GroupChat(agents=[invoice_user, invoice_extraction_agent, 
                                      po_user, po_extraction_agent,
                                      ba_user, ba_extraction_agent,
                                      po_matching_agent, 
                                      reviewer, logger,
                                      Initializer], 
                            messages=[], 
                            max_round=20, 
                            # admin_name="reviewer",  
                            speaker_selection_method="auto"
                            )

system_message = """
# Overview:

invoice_extraction_agent: processes the pdf invoice form given a file path and forwards the extracted data to both po_extraction_agent and po_matching_agent. 

po_extraction_agent: fetches purchase orders for that invoice given a vendor name from the invoice data received from invoice_extraction_agent.

ba_extraction_agent: Recieves bank account balance data and reports to the review agent po_matching_agent to verify the if invoice is covered by the bank account.

po_matching_agent: Recieves invoice data from invoice_extraction_agent,  purchase order data from po_extraction_agent and bank balance from ba_extraction_agent to verify the invoice. It forwards a report to the review agent.

review_agent: Recieves a report from the po_matching_agent. If it finds any report form issues, Reviewer provides detailed feedback with instructions to the po_matching_agent. 
This process will iterate until the report is perfect. REVIEWER CAN PROVIDE FEEBACK TO po_matching_agent.

logger: is skilled at executing python code to store the report.

init: excecutes the python code generated from logger to store the report.


"""
description = """You are a manager of a team of invoice processing experts. 
  invoice_extraction_agent: Reads pdf invoice and extracts all information. 
  po_extraction_agent: Reads purchase orders form a SQL db given a vendor name. 
  ba_extraction_agent: Reads bank account balance information.
  po_matching_agent: Verifies invoice with the PO and creates a report. 
  review_agent: reviews reports
  logger: writes python code"""
manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config=llm_config,
    system_message = system_message,
    description=description

    )



## Simple Crew Version 

Lets create a crew with less member and evelaute the chat performance. 

User Proxy: Runs all the tools   
Evaluation Agent: Does the evaluation task and planning  
Coder Agent: Run code and can log reports

In [30]:
# tools

llm_config_assistant = {
    # "Seed" : 42,
    "temperature": 0,
        "functions":[
            generate_llm_config(invoice_reader_tool),
            generate_llm_config(purchase_order_reader_tool),
            generate_llm_config(bank_account_reader)

        ],
        
    "config_list": config_list,
    "timeout": 120,
}


# Register the tool and start the conversation
function_map={
        invoice_reader_tool.name: invoice_reader_tool._run,
        purchase_order_reader_tool.name: purchase_order_reader_tool._run,
        bank_account_reader.name: bank_account_reader._run,
    }


In [37]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
        code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


user_proxy.register_function(
    function_map = function_map
)

In [40]:
SYSTEM_MESSAGE = """Purchase order matching agent. 

    Solve task using 
    
    You have access to the tools: 'invoice_reader', 
    You receive bank balance data from 'ba_extraction_agent', 'purchase_order_reader' and 'bank_account_reader'

    Your goal is to write a report based on information from all 3 tools.
    
    Vefify that the invoices details below matches the Purchase Order (PO): 
        line_item_net_amount, 
        total_net_amount, 
        gross_amount, 
        tax_amounts, 
        discounts, 
        creditor_name, 
        description_of_goods_or_service, 
        line_items, 
        line_item_amount
    
    Creates a verification report for the PO-invoice match for a CEO:   
    - Overall Status: processing status of the invoice matching 
    - Likelohhod of fraud: number between 1 and 100 that indicates the probability of of fraud as well as the reasonong
    - *Summary*: Summarise the key points.
    - *Red Flags*: Summarise the red flags of missing invoice details in the purchase order  
    - *Check Calculations:* Ensure that all calculations on the invoice, including unit prices, quantities, discounts, and totals, are accurat 
    - Bank account balance coverage: answer can the invoice net amount total be paid with the current bank account balance? 

    The user cannot provide any other feedback or perform any other action. The user can't modify your report. So do not suggest incomplete reports which requires users to modify!

    -----------

    
    Here is an example of the matching report: 
    
        Revised Invoice Verification Report:

        Summary of Findings:
        - Invoice Number: 97159829
        - Creditor Name: Bradley-Andrade
        - Description of Goods or Service: 12" Marble Lapis Inlay Chess Table Top With 2" Pieces & 15" Wooden Stand W537
        - Invoice Date: 09/18/2015
        - Purchase Order Date: 09/18/2015
        - Overall Status: Yellow
        - Likelihood of Fraud: 30% risk (based on discrepancies in tax amounts and missing purchase order number)
        - Bank Account Balance Coverage: No (current balance is $200, invoice net amount is $889.2)
        - Action Items: Contact creditor to resolve tax discrepancy and missing purchase order number; review internal controls.

        Detailed Report:
        - The invoice and purchase order dates align, both being on 09/18/2015.
        - Payment terms are not specified in the provided data for both the invoice and the purchase order.
        - The currency is not mentioned in the provided data; it is assumed to be the local currency of the company.
        - The bank account balance date is 06/23/2024, indicating the financial status at the time of the report.
        - The likelihood of fraud is assessed at 30% risk due to the tax amount discrepancy and the absence of a purchase order number on the invoice. The methodology for this assessment should be reviewed and clarified.
        - The tax amounts are inconsistent, with the invoice stating $88.92 and the purchase order stating $10.00.
        - The purchase order number is missing from the invoice; the corresponding number is 333333.
        - The line item net amount, total net amount, and gross amount match between the invoice and the purchase order.
        - The invoice line item description and amount match the purchase order, with a correction from "2 unite" to "2 units."

        Formatting and Consistency:
        - The report has been formatted for consistency and clarity.

        Next Steps:
        - Contact Bradley-Andrade to address the tax amount discrepancy and to add the missing purchase order number to the invoice.
        - Review internal controls to ensure purchase order numbers are included on all invoices and that tax amounts are correctly calculated and reported.
        - Assess the company's current financial position and take necessary actions to ensure sufficient funds are available to cover upcoming invoices.
            

"""

invoice_processing_agent  = autogen.AssistantAgent(
    name="invoice_processing_agent",
    system_message= SYSTEM_MESSAGE,
    llm_config=llm_config_assistant
)



In [41]:
query = "Process the invoice from the path: .\data\invoices\output_0.pdf" 
user_proxy.initiate_chat(invoice_processing_agent, message=query)

user_proxy (to invoice_processing_agent):

Process the invoice from the path: .\data\invoices\output_0.pdf

--------------------------------------------------------------------------------
invoice_processing_agent (to user_proxy):

***** Suggested function call: invoice_reader *****
Arguments: 
{"path_to_invoice":".\\data\\invoices\\output_0.pdf"}
***************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION invoice_reader...
Started parsing the file under job_id cac11eca-a05a-4290-8399-fdd9a87cbf0d
Function call: Invoice with args: {"date":"09/18/2015","invoice_number":"97159829","net_amount":889.20,"gross_amount":978.12,"tax_amounts":88.92,"seller_name":"Bradley-Andrade","creditor_name":"Castro PLC","description_of_goods_or_service":"12\" Marble Lapis Inlay Chess Table Top With 2\" Pieces & 15\" Wooden Stand W537","line_items":[{"description":"12\" Marble Lapis Inlay Chess Table Top W

# Test

In [125]:
for agent in groupchat.agents:
    agent.reset()

query = "Process the invoice from the path: .\data\invoices\output_0.pdf" 
response = Initializer.initiate_chat(
    manager, 
    message= query,
    summary_method="reflection_with_llm",
    clear_history = True
)


init (to chat_manager):

Process the invoice from the path: .\data\invoices\output_0.pdf

--------------------------------------------------------------------------------

Next speaker: invoice_extraction_agent

invoice_extraction_agent (to chat_manager):

***** Suggested function call: invoice_reader *****
Arguments: 
{"path_to_invoice":".\\data\\invoices\\output_0.pdf"}
***************************************************

--------------------------------------------------------------------------------

Next speaker: invoice_user


>>>>>>>> EXECUTING FUNCTION invoice_reader...
Started parsing the file under job_id cac11eca-4202-47da-93ae-0fbfd2fd2982
Function call: Invoice with args: {"date":"09/18/2015","invoice_number":"97159829","net_amount":889.20,"gross_amount":978.12,"tax_amounts":88.92,"seller_name":"Bradley-Andrade","creditor_name":"Bradley-Andrade","description_of_goods_or_service":"12\" Marble Lapis Inlay Chess Table Top With 2\" Pieces & 15\" Wooden Stand W537","line_items"

In [ ]:
Markdown( response.summary)

********************************
Run without bank account details

In [62]:
for agent in groupchat.agents:
    agent.reset()

query = "Process the invoice from the path: .\data\invoices\output_0.pdf" 
response = Initializer.initiate_chat(
    manager, 
    message= query,
    summary_method="reflection_with_llm",
    clear_history = True
)


init (to chat_manager):

Process the invoice from the path: .\data\invoices\output_0.pdf

--------------------------------------------------------------------------------

Next speaker: invoice_extraction_agent

invoice_extraction_agent (to chat_manager):

***** Suggested function call: invoice_reader *****
Arguments: 
{"path_to_invoice":".\\data\\invoices\\output_0.pdf"}
***************************************************

--------------------------------------------------------------------------------

Next speaker: invoice_user


>>>>>>>> EXECUTING FUNCTION invoice_reader...
Started parsing the file under job_id cac11eca-9945-4214-8066-d8bfff2d6d8c
Function call: Invoice with args: {"date":"09/18/2015","invoice_number":"97159829","net_amount":889.20,"gross_amount":978.12,"tax_amounts":88.92,"seller_name":"Bradley-Andrade","creditor_name":"Bradley-Andrade","description_of_goods_or_service":"12\" Marble Lapis Inlay Chess Table Top With 2\" Pieces & 15\" Wooden Stand W537","line_items"

# Task

Change model to GPT4o
